In [1]:
import numpy as np
import h5py

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import Input, Flatten, Dense, Dropout,  Activation, Conv3D, MaxPooling3D, UpSampling3D, BatchNormalization, Activation, Reshape
from keras.layers import Conv2D, MaxPooling2D
#from keras.utils import multi_gpu_model

from keras.optimizers import SGD
from keras.layers.merge import concatenate, add

#from keras.utils.io_utils import HDF5Matrix
from keras.utils import plot_model
from keras import optimizers

import os
import numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from random import shuffle

In [2]:
def conv3d_block(input_tensor, filters, kernel_size, activation_fun = "linear"):
    # first layer
    x = Conv3D(filters=filters, kernel_size=kernel_size, kernel_initializer="he_normal", padding="same")(input_tensor)
    x = BatchNormalization()(x)
    x = Activation(activation_fun)(x)

    return x

kernel_size = (5,5,5)
act_fun = "relu"

############### DEFINE SET OF INPUTS #################

x = Input(shape=(8, 256, 256,1)) 


############## DEFINE PATH FOR INPUT 1 ##################


c1 = conv3d_block (x, filters=8, kernel_size=kernel_size, activation_fun = act_fun )
c1 = BatchNormalization()(c1)
p1 = MaxPooling3D (pool_size=(4, 8, 8), padding='valid')(c1)
p1 = Dropout(0.5)(p1)


c2 = conv3d_block(p1, filters=16, kernel_size=kernel_size, activation_fun = act_fun )
c2 = BatchNormalization()(c2)
p2 = MaxPooling3D(pool_size=(2, 4, 4), padding='valid')(c2)
p2 = Dropout(0.25)(p2)


c3 = conv3d_block(p2, filters=32, kernel_size=kernel_size, activation_fun = act_fun )
c3 = BatchNormalization()(c3)
p3 = MaxPooling3D(pool_size=(1, 2, 2), padding='valid')(c3)
p3 = Dropout(0.25)(p3)


p3 = Reshape((512,))(p3)


d = Dense(2, activation = 'sigmoid')(p3)



model = Model(x , d)
model.compile(optimizer="Adam", loss=tf.keras.losses.BinaryCrossentropy())
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8, 256, 256, 1)]  0         
_________________________________________________________________
conv3d (Conv3D)              (None, 8, 256, 256, 8)    1008      
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 256, 256, 8)    32        
_________________________________________________________________
activation (Activation)      (None, 8, 256, 256, 8)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 256, 256, 8)    32        
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 2, 32, 32, 8)      0         
_________________________________________________________________
dropout (Dropout)            (None, 2, 32, 32, 8)      0     

In [3]:
class DataGenerator(keras.utils.Sequence):

    def __init__(self, Directorio, batch_size = 5):
        
        self.batch_size = batch_size
        self.Directorio = h5py.File(Directorio)

    def __len__(self):
        
        return int((len(self.Directorio)/2) / self.batch_size)

    
    #Getitem nos devolvera los valores de X e Y para el indice de batch que le pida keras
    def __getitem__(self, index):
        
        #Creamos dos listas vacias en las que se separaran los valores de X e Y de nuestro archivo
        array_x = []
        array_y = []
        
        #Numero de ejemplos que tenemos en nuestro dataset, dividimos entre dos por que la longitud total
        #viene con el numero de X e Y
        self.indexes = np.arange(int(len(self.Directorio)/2))
        
        #dependiendo del indice de batch, indexes nos dara la posicion de los datos que queremos extraer
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        #Recorremos el directorio y extraemos las X con sus frames y las Y con sus estados
        for line in self.Directorio:

            if line[0] == 'X':
                array_x.append(self.Directorio[line])

            if line [0] == 'Y':
                array_y.append(self.Directorio[line])
        
        #dos nuevos listas en las que almacenaremos los datos correspondientes a las posiciones que nos pide
        #el indice del batch
        x = []
        y = []
        
        for i in indexes:
            x.append(array_x[i])
            y.append(array_y[i])
        
        #convertimos la lista anterior en un array con shape (batch_size,8,512,512,3)
        x_array = np.array(x)
        
        
        #Keras no entiende de strings, por lo que pasamos esos strings a vectores con numeros
        RE = [1,0] #vector para estado: reposo
        EA = [0,1] #vector para estado: ataque
        
        #lista donde almacenamos los vectores asociados a su string correspondiente
        y_strings = []
        
        for i in y:

            if i.value == 'RE':
                y_strings.append(RE)

            else:
                y_strings.append(EA)
                
        #convertimos la lista anterior en un array con shape (batch_size,2)
        y_array = np.array(y_strings)

        X, y = x_array,y_array

        return X,y

batch_size = 200


train_samples =  '/mnt/MD1200A/lconcha/videos/256x256/Datasets_training_256/Combinado_training_group32_256_blackwhite'
test_samples =  '/mnt/MD1200A/lconcha/videos/256x256/Datasets_training_256/Combinado_validation_group32_256_2_blackwhite'

training_generator   = DataGenerator(train_samples, batch_size )
validation_generator = DataGenerator(test_samples,  batch_size )

/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  


In [ ]:
model.compile(optimizer="Adam", loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])


#####CHECK POINTS
checkpoint_path = "/mnt/MD1200A/lconcha/videos/Modelos/256x256_blackwhite_dropout_00500250025/training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 period = 10,
                                                 verbose=1)


history = model.fit_generator(generator=training_generator,
                    steps_per_epoch= int((len(h5py.File(train_samples))/2))// batch_size,
                    epochs=50,
                    callbacks=[cp_callback],
                    verbose=True,
                    validation_data=validation_generator,
                    validation_steps=int((len(h5py.File(test_samples))/2))// batch_size,
                    use_multiprocessing=False,
                    workers=0,
                    max_queue_size=5)

/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  app.launch_new_instance()
/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:21: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site

Epoch 1/50
15/15 [==============================] - 2408s 160s/step - loss: 0.9378 - accuracy: 0.5474 - val_loss: 1.4303 - val_accuracy: 0.5130
Epoch 2/50
15/15 [==============================] - 2406s 160s/step - loss: 0.7864 - accuracy: 0.6122 - val_loss: 0.8487 - val_accuracy: 0.5780
Epoch 3/50
15/15 [==============================] - 2394s 160s/step - loss: 0.7442 - accuracy: 0.6262 - val_loss: 0.7228 - val_accuracy: 0.5860
Epoch 4/50
15/15 [==============================] - 2384s 159s/step - loss: 0.7109 - accuracy: 0.6595 - val_loss: 0.6576 - val_accuracy: 0.6550
Epoch 5/50
15/15 [==============================] - 2400s 160s/step - loss: 0.6869 - accuracy: 0.6555 - val_loss: 0.6793 - val_accuracy: 0.6010
Epoch 6/50
15/15 [==============================] - 2409s 161s/step - loss: 0.6711 - accuracy: 0.6654 - val_loss: 0.6618 - val_accuracy: 0.6240
Epoch 7/50
15/15 [==============================] - 2405s 160s/step - loss: 0.6465 - accuracy: 0.6859 - val_loss: 0.6332 - val_accuracy:

In [4]:
checkpoint_path = "/mnt/MD1200A/lconcha/videos/Modelos/256x256_blackwhite_dropout_00500250025/training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
model.load_weights(checkpoint_path)

In [5]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 period = 10,
                                                 verbose=1)


history = model.fit_generator(generator=training_generator,
                    steps_per_epoch= int((len(h5py.File(train_samples))/2))// batch_size,
                    epochs=10,
                    callbacks=[cp_callback],
                    verbose=True,
                    validation_data=validation_generator,
                    validation_steps=int((len(h5py.File(test_samples))/2))// batch_size,
                    use_multiprocessing=False,
                    workers=0,
                    max_queue_size=5)

/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:9: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  if __name__ == '__main__':
/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  
/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/s

Epoch 1/10
15/15 [==============================] - 2175s 145s/step - loss: 0.4936 - val_loss: 0.4923
Epoch 2/10
15/15 [==============================] - 2149s 143s/step - loss: 0.5097 - val_loss: 0.4962
Epoch 3/10
15/15 [==============================] - 2145s 143s/step - loss: 0.4917 - val_loss: 0.5088
Epoch 4/10
15/15 [==============================] - 2147s 143s/step - loss: 0.4934 - val_loss: 0.6355
Epoch 5/10
15/15 [==============================] - 2165s 144s/step - loss: 0.4836 - val_loss: 0.6208
Epoch 6/10
15/15 [==============================] - 2157s 144s/step - loss: 0.4850 - val_loss: 0.5930
Epoch 7/10
15/15 [==============================] - 2148s 143s/step - loss: 0.4889 - val_loss: 0.5935
Epoch 8/10
15/15 [==============================] - 2145s 143s/step - loss: 0.4851 - val_loss: 0.5531
Epoch 9/10
15/15 [==============================] - 2193s 147s/step - loss: 0.4755 - val_loss: 0.5276
Epoch 10/10
15/15 [==============================] - 2199s 147s/step - loss: 0.471

In [8]:
model.history.history['val_loss']

AttributeError: 'NoneType' object has no attribute 'history'